##Import library

In [9]:
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np
from tqdm.notebook import tqdm
import re
from functools import partial
from scipy.stats import mode
import matplotlib.pyplot as plt; plt.style.use('ggplot')
import seaborn as sns
import plotly.express as px
from sklearn.linear_model import Lasso, LinearRegression, Ridge, ElasticNet
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder, FunctionTransformer, PowerTransformer, PolynomialFeatures, RobustScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer, SimpleImputer, IterativeImputer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GridSearchCV, RepeatedStratifiedKFold, cross_val_score, cross_val_predict, RepeatedKFold
from sklearn.metrics import roc_auc_score, roc_curve, RocCurveDisplay, cohen_kappa_score, log_loss, f1_score, median_absolute_error, accuracy_score, mean_squared_error
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.feature_selection import RFE, RFECV
from sklearn.isotonic import IsotonicRegression
from sklearn.calibration import CalibrationDisplay
from sklearn.inspection import PartialDependenceDisplay, permutation_importance
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor, GradientBoostingRegressor, ExtraTreesRegressor, VotingRegressor, StackingRegressor, BaggingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVR
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from tensorflow.keras.models import Sequential
from keras.callbacks import EarlyStopping, TerminateOnNaN
from scikeras.wrappers import KerasRegressor
import optuna
import tensorflow as tf
from tensorflow import keras
import warnings
warnings.filterwarnings('ignore')
sns.set()
import matplotlib.pyplot as plt
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin,clone
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
import cv2

In [10]:
import cv2

##Import Data

In [2]:
label = pd.read_csv("label.csv")
sample = pd.read_csv("sample_submission.csv")

In [3]:
label.head()

,tps,suara_paslon_1,suara_paslon_2,suara_paslon_3
0,TPS_001,159,9,2
1,TPS_002,112,87,2
2,TPS_003,173,27,3
3,TPS_004,207,20,3
4,TPS_005,213,30,4


In [4]:
sample.head(100)

,TPS,Persentase Suara Sah Paslon 2
0,TPS_501,NaN
1,TPS_502,NaN
2,TPS_503,NaN
3,TPS_504,NaN
4,TPS_505,NaN
...,...,...
95,TPS_596,NaN
96,TPS_597,NaN
97,TPS_598,NaN
98,TPS_599,NaN


In [11]:
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")

def process_images_yolo(image_path):
    img = cv2.imread(image_path)
    height, width, channels = img.shape

    part_height = height // 3
    parts = [img[i*part_height:(i+1)*part_height, :] for i in range(3)]

    results = []
    for part in parts:
        part = cv2.resize(part, None, fx=0.4, fy=0.4)
        height, width, channels = part.shape

        blob = cv2.dnn.blobFromImage(part, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        outs = net.forward(output_layers)

        class_ids = []
        confidences = []
        boxes = []
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

        labels = []
        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                label = str(classes[class_ids[i]])
                labels.append(label)
        
        results.append(max(set(labels), key=labels.count) if labels else None)

    return results

df = pd.DataFrame(columns=['tps', 'suara_paslon_1', 'suara_paslon_2', 'suara_paslon_3'])

for i in range(1, 501):
    image_path = f"Train/{i:03d}.jpg"
    results = process_images_yolo(image_path)
    df.loc[i] = [f"{i:03d}"] + results

# df.to_csv("results.csv", index=False)

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\darknet\darknet_importer.cpp:210: error: (-212:Parsing error) Failed to open NetParameter file: yolov3.cfg in function 'cv::dnn::dnn4_v20231225::readNetFromDarknet'
